In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración estética para los gráficos
# 'whitegrid' ayuda a leer mejor los valores con líneas de fondo
sns.set_style("whitegrid")
# Definimos un tamaño de gráfico estándar para todo el notebook
plt.rcParams['figure.figsize'] = (10, 6)

def cargar_datos():
    """
    Tarea 1: Carga y Limpieza de Datos.
    
    Esta función lee los archivos CSV necesarios para el análisis.
    También realiza un chequeo rápido de valores nulos para asegurar la calidad de los datos.
    
    Returns:
        tuple: Contiene los 4 DataFrames (clientes, detalle, productos, ventas).
    """
    try:
        # Carga de archivos CSV
        df_cli = pd.read_csv('clientes.xlsx - clientes.csv.csv')
        df_det = pd.read_csv('detalle_ventas.xlsx - detalle_ventas.csv.csv')
        df_prod = pd.read_csv('productos.xlsx - productos.csv.csv')
        df_ven = pd.read_csv('ventas.xlsx - ventas.csv.csv')
        
        print("✅ Fase 1: Datos cargados exitosamente.\n")
        
        # Verificación de integridad de datos (Búsqueda de nulos)
        print("--- Reporte de Valores Nulos ---")
        datasets = {'Clientes': df_cli, 'Detalle': df_det, 'Productos': df_prod, 'Ventas': df_ven}
        for nombre, df in datasets.items():
            nulos = df.isnull().sum().sum()
            estado = "Sin nulos" if nulos == 0 else f"⚠️ {nulos} nulos detectados"
            print(f"{nombre}: {estado}")
            
        return df_cli, df_det, df_prod, df_ven
    except FileNotFoundError as e:
        print(f"❌ Error crítico: No se encontró el archivo. {e}")
        return None, None, None, None
    
    def crear_dataset_maestro(df_ven, df_det, df_prod, df_cli):
    """
    Tarea 2: Fusión de Datos (Data Merging).
    
    Crea una única tabla 'Maestra' que consolida toda la información dispersa.
    La lógica de unión es:
    1. Ventas + Detalle (Unión 1 a N): Una venta tiene muchos productos.
    2. + Productos: Para saber el nombre y categoría de lo vendido.
    3. + Clientes: Para saber quién compró y de dónde es.
    
    Args:
        df_ven, df_det, df_prod, df_cli: DataFrames individuales cargados previamente.
        
    Returns:
        pd.DataFrame: DataFrame consolidado y listo para análisis.
    """
    # Paso 1: Unir cabecera de factura con detalle de items
    print("\n🔄 Procesando fusión de tablas...")
    master = pd.merge(df_ven, df_det, on='id_venta', how='inner')
    
    # Paso 2: Enriquecer con datos del producto
    master = pd.merge(master, df_prod, on='id_producto', how='left')
    
    # Paso 3: Enriquecer con datos del cliente
    # Usamos suffixes para evitar conflictos si hay columnas con el mismo nombre (ej. email)
    master = pd.merge(master, df_cli, on='id_cliente', how='left', suffixes=('', '_cliente'))
    
    # Conversión de tipos: La fecha debe ser objeto datetime para análisis temporal
    master['fecha'] = pd.to_datetime(master['fecha'])
    
    print(f" Fase 2 Completada. Dimensiones del Dataset Maestro: {master.shape}")
    return master